# Practica 3 sentiment

## Importar

In [50]:
# %load basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [51]:
liga = '/home/ef/Documents/Diplomado/data/'
df = pd.read_csv(liga + 'sentiment.csv')
df.sample(4)

,review,sentiment
5166,Oh dear! The BBC is not about to be knocked of...,negative
27458,"I won't bore you with story and plot lines, as...",positive
9187,"Got into this flick, just as it was beginning,...",negative
35930,Superbly trashy and wondrously unpretentious 8...,positive


In [52]:
df.shape

(50000, 2)

## Ingeniería de variables

In [53]:
import re
import unicodedata
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().split())
    return cleaned_text

In [54]:
df.sample()

,review,sentiment
16509,I heard this film was much more stylistic than...,negative


In [55]:
df['limpio'] = [clean_text(x) for x in df['review']]
df.sample()

,review,sentiment,limpio
4228,"A letter to the guys. I tried guys, I really t...",positive,a letter to the guys i tried guys i really tri...


In [56]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

texto = []
for x in [n.split() for n in df['limpio']]:
    aux = []
    for word in x:
        if word not in stop_words:
            aux.append(word)
    texto.append(aux)
            
df['limpio_stop'] = [" ".join(x) for x in texto]
df.sample(2)

[nltk_data] Downloading package stopwords to /home/ef/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment,limpio,limpio_stop
43637,"""The Last Wave"" is one of those movies that re...",positive,the last wave is one of those movies that reli...,last wave one movies relies heavily mind title...
1185,The club scenes in this film are extremely bel...,positive,the club scenes in this film are extremely bel...,club scenes film extremely believable tim curr...


In [57]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(1, 1), 
                     min_df=10, 
                     max_features=100)
cv.fit(df['limpio_stop'])
df = df.join(pd.DataFrame(data = cv.transform(df['limpio_stop']).todense(),
                          columns = cv.get_feature_names()))

words = list(df.iloc[:,4:].columns)
df.sample(2)

,review,sentiment,limpio,limpio_stop,10,acting,actors,actually,also,another,...,want,watch,watching,way,well,work,world,would,years,young
1409,Feels like an impressionistic film; if there i...,positive,feels like an impressionistic film if there is...,feels like impressionistic film thing br br st...,0.0,0.000000,0.122631,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.461301,0.0,0.0,0.000000,0.0,0.0
47108,"I, like so many others on here, bought this mo...",negative,i like so many others on here bought this movi...,like many others bought movie local wm two 11 ...,0.0,0.188466,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.157782,0.0,0.0


In [58]:
df['len_text_raw'] = df['review'].str.len()
df['n_lower_ratio_len'] = df['review'].map(lambda x:sum(map(str.islower, x))) / df['len_text_raw']
df['n_words'] = df['review'].str.split(' ').str.len()
df['n_letters'] = df['review'].map(lambda x:sum(map(str.isalpha, x)))
df[[x for x in df.columns if x not in words]].sample(2)

,review,sentiment,limpio,limpio_stop,len_text_raw,n_lower_ratio_len,n_words,n_letters
8283,I wasn't sure what to expect but am I glad I w...,positive,i wasn t sure what to expect but am i glad i w...,sure expect glad went see smart slightly twist...,429,0.731935,82,327
26620,"As usual, i went to watch this movie for A.R.R...",negative,as usual i went to watch this movie for a r ra...,usual went watch movie r rahman otherwise film...,715,0.713287,134,535


In [59]:
df['target'] = np.where(df['sentiment'] == 'negative', 1, 0)
df['target'].mean()

0.5

In [60]:
df[[x for x in df.columns if x not in words]].describe()

,len_text_raw,n_lower_ratio_len,n_words,n_letters,target
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,1309.431020,0.752438,231.145940,1023.344260,0.500000
std,989.728014,0.026092,171.326419,777.784587,0.500005
min,32.000000,0.000000,4.000000,25.000000,0.000000
25%,699.000000,0.739583,126.000000,544.000000,0.000000
50%,970.000000,0.755474,173.000000,757.000000,0.500000
75%,1590.250000,0.769118,280.000000,1243.000000,1.000000
max,13704.000000,0.826568,2470.000000,10862.000000,1.000000


## Modelado

In [61]:
df.sample(2)

,review,sentiment,limpio,limpio_stop,10,acting,actors,actually,also,another,...,work,world,would,years,young,len_text_raw,n_lower_ratio_len,n_words,n_letters,target
19836,THIS REVIEW IS MOSTLY ALL SPOILERS. IF YOU PLA...,negative,this review is mostly all spoilers if you plan...,review mostly spoilers plan enjoying film read...,0.0,0.000000,0.0,0.0,0.000000,0.079266,...,0.0,0.085803,0.171865,0.317926,0.000000,4213,0.732020,751,3276,1
2019,Forest of the Damned starts out as five young ...,negative,forest of the damned starts out as five young ...,forest damned starts five young friends brothe...,0.0,0.151844,0.0,0.0,0.070671,0.000000,...,0.0,0.000000,0.000000,0.000000,0.096941,3423,0.756062,614,2672,1


In [83]:
X = df.iloc[:,4:-1]
y = df.iloc[:,-1:]
y.mean()

target    0.5
dtype: float64

In [85]:
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

from sklearn.preprocessing import MinMaxScaler
mm_x = MinMaxScaler()
mm_y = MinMaxScaler()
Xs = pd.DataFrame(mm_x.fit_transform(X_train),
                 columns = X.columns)
ys = mm_y.fit_transform(y_train).ravel()
Xs_test = pd.DataFrame(mm_x.transform(X_test),
                 columns = X.columns)
ys_test = mm_y.transform(y_test).ravel()

Xs.describe()

,10,acting,actors,actually,also,another,around,back,bad,best,...,well,work,world,would,years,young,len_text_raw,n_lower_ratio_len,n_words,n_letters
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,...,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.028828,0.043675,0.033539,0.034177,0.046148,0.034176,0.031569,0.034295,0.050895,0.035872,...,0.054209,0.030198,0.023695,0.061303,0.029967,0.025602,0.092993,0.895222,0.092263,0.091859
std,0.084749,0.096415,0.093485,0.097131,0.093143,0.095106,0.097494,0.092121,0.113431,0.086885,...,0.099841,0.087660,0.077240,0.108198,0.084915,0.085986,0.072790,0.036495,0.069796,0.072163
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048086,0.877023,0.049473,0.047452
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067921,0.899486,0.068532,0.066977
75%,0.000000,0.000000,0.000000,0.000000,0.054926,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.094468,0.000000,0.000000,0.111092,0.000000,0.000000,0.113591,0.918703,0.112328,0.112168
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [86]:
Xs_test.sample(4)

,10,acting,actors,actually,also,another,around,back,bad,best,...,well,work,world,would,years,young,len_text_raw,n_lower_ratio_len,n_words,n_letters
2584,0.081956,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,...,0.000000,0.00000,0.337812,0.000000,0.000000,0.0,0.308058,0.935152,0.276561,0.309915
8902,0.000000,0.0,0.137999,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.104780,...,0.000000,0.00000,0.000000,0.097613,0.131144,0.0,0.085999,0.933596,0.084753,0.086688
9457,0.000000,0.0,0.000000,0.232811,0.000000,0.0,0.00000,0.0,0.0,0.149136,...,0.137765,0.39615,0.000000,0.138935,0.000000,0.0,0.101954,0.861662,0.103812,0.101182
1346,0.000000,0.0,0.622982,0.000000,0.155181,0.0,0.27435,0.0,0.0,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.052477,0.924380,0.062044,0.051883


In [87]:
from sklearn.feature_selection import SelectKBest, f_classif
kb = SelectKBest(k = 'all', score_func = f_classif)
kb.fit(Xs, ys.ravel())
ls_best = [x for x, y in zip(Xs.columns, kb.get_support()) if y]

In [88]:
Xs = Xs[ls_best]
Xs.describe()

,10,acting,actors,actually,also,another,around,back,bad,best,...,well,work,world,would,years,young,len_text_raw,n_lower_ratio_len,n_words,n_letters
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,...,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.028828,0.043675,0.033539,0.034177,0.046148,0.034176,0.031569,0.034295,0.050895,0.035872,...,0.054209,0.030198,0.023695,0.061303,0.029967,0.025602,0.092993,0.895222,0.092263,0.091859
std,0.084749,0.096415,0.093485,0.097131,0.093143,0.095106,0.097494,0.092121,0.113431,0.086885,...,0.099841,0.087660,0.077240,0.108198,0.084915,0.085986,0.072790,0.036495,0.069796,0.072163
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048086,0.877023,0.049473,0.047452
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067921,0.899486,0.068532,0.066977
75%,0.000000,0.000000,0.000000,0.000000,0.054926,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.094468,0.000000,0.000000,0.111092,0.000000,0.000000,0.113591,0.918703,0.112328,0.112168
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Regresión Logística

In [89]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight = 'balanced')
logreg.fit(Xs, ys)
ls_scores = cross_val_score(X = Xs,
                            y = ys,
                            cv = 3,
                            estimator = logreg,
                            scoring = 'roc_auc')
## Se observa la acertividad en diferentes secciones del conjunto de entrenamiento
np.mean(ls_scores), np.std(ls_scores)

(0.8093865911819597, 0.0024130908906658762)

#### Hiper-parametrización

In [90]:
param_grid = {"penalty": ["l1", "l2", "elasticnet"],
              "C": [x/500 for x in range(500)]+[0],
              "class_weight": ["balanced", None],
              "solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
             }
np.prod(list(map(len, param_grid.values())))

15030

In [91]:
from sklearn.model_selection import RandomizedSearchCV
hp = RandomizedSearchCV(cv = 3, 
                        param_distributions = param_grid,
                        n_iter = 44,
                        scoring = 'roc_auc', 
                        verbose = 1,
                        error_score = -10000, 
                        estimator = logreg, 
                        n_jobs = -1)
hp.fit(X = Xs, y = ys)
hp.best_score_

Fitting 3 folds for each of 44 candidates, totalling 132 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 132 out of 132 | elapsed:  5.6min finished


0.8094208798490313

### Prueba

In [93]:
## Y se corre para el conjunto de prueba
from sklearn.metrics import roc_auc_score
'{:,.2%}'.format(roc_auc_score(y_score = logreg.predict(Xs_test[ls_best]),
                              y_true = ys_test))

'73.09%'

In [94]:
## Para hp randomized
'{:,.2%}'.format(roc_auc_score(y_score = hp.predict(Xs_test[ls_best]),
                              y_true = ys_test))

'73.11%'

In [96]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_pred = hp.predict(Xs_test[ls_best]),
                              y_true = ys_test) / confusion_matrix(y_pred = hp.predict(Xs_test[ls_best]),
                                                                   y_true = ys_test).sum())

,0,1
0,0.3715,0.1369
1,0.1320,0.3596


In [97]:
## Se despliega la relación que tiene cada una de las mejores variables que explican
# la variable objetivo
pd.DataFrame(list(zip(ls_best, hp.best_estimator_.coef_[0])),
            columns = ['Variable', 'Explica']).sort_values(by = 'Explica')

,Variable,Explica
36,great,-4.139401
9,best,-3.300950
46,love,-2.503591
101,n_lower_ratio_len,-2.460101
94,well,-2.348763
...,...,...
82,thing,2.014144
64,plot,2.228062
21,even,2.415777
59,nothing,3.544646
